
<h1 id="Developing-an-AI-application">Developing an AI application<a class="anchor-link" href="#Developing-an-AI-application">¶</a></h1><p>Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications.</p>
<p>In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using <a href="http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html">this dataset</a> of 102 flower categories, you can see a few examples below.</p>
<p><img src="assets/Flowers.png" width="500px"/></p>
<p>The project is broken down into multiple steps:</p>
<ul>
<li>Load and preprocess the image dataset</li>
<li>Train the image classifier on your dataset</li>
<li>Use the trained classifier to predict image content</li>
</ul>
<p>We'll lead you through each part which you'll implement in Python.</p>
<p>When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.</p>
<p>First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.</p>


In [ ]:

# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from workspace_utils import active_session
from collections import OrderedDict 
from PIL import Image
import json




<h2 id="Load-the-data">Load the data<a class="anchor-link" href="#Load-the-data">¶</a></h2><p>Here you'll use <code>torchvision</code> to load the data (<a href="http://pytorch.org/docs/0.3.0/torchvision/index.html">documentation</a>). The data should be included alongside this notebook, otherwise you can <a href="https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz">download it here</a>. The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.</p>
<p>The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.</p>
<p>The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's <code>[0.485, 0.456, 0.406]</code> and for the standard deviations <code>[0.229, 0.224, 0.225]</code>, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.</p>


In [ ]:

data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'



In [ ]:

# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])]) 

valid_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])]) 

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=50, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=25, shuffle = False)
testloader = torch.utils.data.DataLoader(test_data, batch_size=25, shuffle = False)



In [ ]:

print(f"trainining data len: {len(train_data)}")
print(f"validation data len: {len(valid_data)}")
print(f"test data len: {len(test_data)}")
print(f"training loader len: {len(trainloader)}")
print(f"validation loader len: {len(validloader)}")
print(f"testing loader len: {len(testloader)}")




<h3 id="Label-mapping">Label mapping<a class="anchor-link" href="#Label-mapping">¶</a></h3><p>You'll also need to load in a mapping from category label to category name. You can find this in the file <code>cat_to_name.json</code>. It's a JSON object which you can read in with the <a href="https://docs.python.org/2/library/json.html"><code>json</code> module</a>. This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.</p>


In [ ]:

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)




<h1 id="Building-and-training-the-classifier">Building and training the classifier<a class="anchor-link" href="#Building-and-training-the-classifier">¶</a></h1><p>Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from <code>torchvision.models</code> to get the image features. Build and train a new feed-forward classifier using those features.</p>
<p>We're going to leave this part up to you. Refer to <a href="https://review.udacity.com/#!/rubrics/1663/view">the rubric</a> for guidance on successfully completing this section. Things you'll need to do:</p>
<ul>
<li>Load a <a href="http://pytorch.org/docs/master/torchvision/models.html">pre-trained network</a> (If you need a starting point, the VGG networks work great and are straightforward to use)</li>
<li>Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout</li>
<li>Train the classifier layers using backpropagation using the pre-trained network to get the features</li>
<li>Track the loss and accuracy on the validation set to determine the best hyperparameters</li>
</ul>
<p>We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!</p>
<p>When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.</p>
<p>One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.</p>
<p><strong>Note for Workspace users:</strong> If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with <code>ls -lh</code>), you should reduce the size of your hidden layers and train again.</p>


In [ ]:

# Create VGG model, freeze parameter & define classifier
def create_model(architecture='vgg'):
    
    if architecture == 'vgg':
        model = models.vgg16(pretrained=True)
        n_input = 25088
    elif architecture == 'densenet':
        model = models.densenet121(pretrained=True)
        n_input = 1024
    else: 
        raise Exception("Unknown structure. Please use 'vgg' or 'densenet' structure.")
    
    # Freeze parameters so we don't backpropagate through them
    for param in model.parameters():
        param.requires_grad = False
    print('Grad parameters freezed.')
    
    # Define classifier
    classifier = nn.Sequential()
    classifier.add_module("inputs", nn.Linear(n_input, 4096, bias=True))
    classifier.add_module("bn1", nn.BatchNorm1d(4096))
    classifier.add_module("relu1", nn.ReLU())
    classifier.add_module("dropout1", nn.Dropout(0.5))                         
    classifier.add_module("hidden_layer1", nn.Linear(4096, 512))
    classifier.add_module("bn2", nn.BatchNorm1d(512))
    classifier.add_module("relu2", nn.ReLU())
    classifier.add_module("dropout2", nn.Dropout(0.5))                              
    classifier.add_module("hidden_layer2", nn.Linear(512, 102, bias=True))
    classifier.add_module("outputs", nn.LogSoftmax(dim=1))    
    model.classifier = classifier
                          
    print(f'Model ({architecture}) created.')
    
    return model



In [ ]:

# Get device
def get_device():
     return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [ ]:

# Evaluation function
def evaluate(model, criterion, loader):
    device = get_device()
    
    #if torch.cuda.is_available() == False:
    #    raise Exception('GPU must be available.')
    
    eval_loss = 0
    accuracy = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)
            
            batch_loss = criterion(logps, labels)       
            eval_loss += batch_loss.item()
            
            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
    
    return eval_loss, accuracy



In [ ]:

# Train the model
def train(model, criterion, optimizer):
    print('Training started...')
    
    epochs = 10
    steps = 0
    running_loss = 0
    eval_every_step = 25
    for epoch in range(epochs):
        for inputs, labels in trainloader:
            steps += 1
            # Move input and label tensors to the default device
            inputs, labels = inputs.to(device), labels.to(device)
        
            optimizer.zero_grad()
        
            logps = model.forward(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
            # Validate
            if steps % eval_every_step == 0:              
                test_loss, accuracy = evaluate(model, criterion, validloader)   
                     
                # Output
                print(f"Epoch {epoch+1}/{epochs}.. "
                    f"Train loss: {running_loss/eval_every_step:.3f}.. "
                    f"Validation loss: {test_loss/len(validloader):.3f}.. "
                    f"Validation accuracy: {accuracy/len(validloader):.3f}")                  
                
                running_loss = 0
                model.train()
            
    print('Training finished.')



In [ ]:

#Create & prepare model
model = create_model('densenet')
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
device = get_device()
model.to(device)

print(model)



In [ ]:

#Start training
with active_session():  
    train(model, criterion, optimizer)




<h2 id="Testing-your-network">Testing your network<a class="anchor-link" href="#Testing-your-network">¶</a></h2><p>It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.</p>


In [ ]:

test_loss, accuracy = evaluate(model, criterion, testloader)

print(f"Testing loss: {test_loss/len(testloader):.3f}.. "
      f"Testing accuracy: {accuracy/len(testloader):.3f}")    




<h2 id="Save-the-checkpoint">Save the checkpoint<a class="anchor-link" href="#Save-the-checkpoint">¶</a></h2><p>Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: <code>image_datasets['train'].class_to_idx</code>. You can attach this to the model as an attribute which makes inference easier later on.</p>
<p><code>model.class_to_idx = image_datasets['train'].class_to_idx</code></p>
<p>Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, <code>optimizer.state_dict</code>. You'll likely want to use this trained model in the next part of the project, so best to save it now.</p>


In [ ]:

# TODO: Save the checkpoint 

def save_model(model, architecture):
    print('Model checkpoint is saving...')
    model.class_to_idx = train_data.class_to_idx
    checkpoint = {'architecture': architecture,
                  'classifier': model.classifier,
                  'state_dict': model.state_dict(),
                  'optimizer': optimizer.state_dict(),
                  'class_to_idx': model.class_to_idx}
    torch.save(checkpoint, 'checkpoint-10e.pth')
    print('Model checkpoint saved.')



In [ ]:

save_model(model, 'densenet')




<h2 id="Loading-the-checkpoint">Loading the checkpoint<a class="anchor-link" href="#Loading-the-checkpoint">¶</a></h2><p>At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.</p>


In [ ]:

# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_model():
    
    print('Model is loading...')
    
    #Perform load with the map_location parameter
    if torch.cuda.is_available():
        map_location=lambda storage, loc: storage.cuda()
    else:
        map_location='cpu'

    checkpoint = torch.load('checkpoint.pth', map_location=map_location)
    
    # Create pretrained model (architecture dependent)
    architecture = checkpoint['architecture']
    if architecture == 'vgg':
        model = models.vgg16(pretrained=True)
    elif architecture == 'densenet':
        model = models.densenet121(pretrained=True)
    else:
        raise Exception(f'Unknown architecture {architecture}. Cannot restore the model.')
    
    # Update model
    model.classifier = checkpoint['classifier']
    model.class_to_idx = checkpoint['class_to_idx']
    model.load_state_dict(checkpoint['state_dict'])   
    
    # Freeze parameters 
    for param in model.parameters():
        param.requires_grad = False
   
    model.eval()

    print('Model loaded from checkpoint.')
    
    return model



In [ ]:

model = load_model()
model.to(get_device())



In [ ]:

# Test loaded model
criterion = nn.NLLLoss()
test_loss, accuracy = evaluate(model, criterion, testloader)

print(f"Testing loss: {test_loss/len(testloader):.3f}.. "
      f"Testing accuracy: {accuracy/len(testloader):.3f}")  




<h1 id="Inference-for-classification">Inference for classification<a class="anchor-link" href="#Inference-for-classification">¶</a></h1><p>Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called <code>predict</code> that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like</p>
<div class="highlight"><pre><span></span><span class="n">probs</span><span class="p">,</span> <span class="n">classes</span> <span class="o">=</span> <span class="n">predict</span><span class="p">(</span><span class="n">image_path</span><span class="p">,</span> <span class="n">model</span><span class="p">)</span>
<span class="k">print</span><span class="p">(</span><span class="n">probs</span><span class="p">)</span>
<span class="k">print</span><span class="p">(</span><span class="n">classes</span><span class="p">)</span>
<span class="o">&gt;</span> <span class="p">[</span> <span class="mf">0.01558163</span>  <span class="mf">0.01541934</span>  <span class="mf">0.01452626</span>  <span class="mf">0.01443549</span>  <span class="mf">0.01407339</span><span class="p">]</span>
<span class="o">&gt;</span> <span class="p">[</span><span class="s1">'70'</span><span class="p">,</span> <span class="s1">'3'</span><span class="p">,</span> <span class="s1">'45'</span><span class="p">,</span> <span class="s1">'62'</span><span class="p">,</span> <span class="s1">'55'</span><span class="p">]</span>
</pre></div>
<p>First you'll need to handle processing the input image such that it can be used in your network.</p>
<h2 id="Image-Preprocessing">Image Preprocessing<a class="anchor-link" href="#Image-Preprocessing">¶</a></h2><p>You'll want to use <code>PIL</code> to load the image (<a href="https://pillow.readthedocs.io/en/latest/reference/Image.html">documentation</a>). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training.</p>
<p>First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the <a href="http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail"><code>thumbnail</code></a> or <a href="http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail"><code>resize</code></a> methods. Then you'll need to crop out the center 224x224 portion of the image.</p>
<p>Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so <code>np_image = np.array(pil_image)</code>.</p>
<p>As before, the network expects the images to be normalized in a specific way. For the means, it's <code>[0.485, 0.456, 0.406]</code> and for the standard deviations <code>[0.229, 0.224, 0.225]</code>. You'll want to subtract the means from each color channel, then divide by the standard deviation.</p>
<p>And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using <a href="https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html"><code>ndarray.transpose</code></a>. The color channel needs to be first and retain the order of the other two dimensions.</p>


In [ ]:

def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
  
    preprocess = transforms.Compose([
                               transforms.Resize(256),
                               transforms.CenterCrop(224),
                               transforms.ToTensor(),
                               transforms.Normalize(
                                               [0.485, 0.456, 0.406],
                                               [0.229, 0.224, 0.225])])
    
    img_pil = Image.open(image)
    img_tensor = preprocess(img_pil)
    
    return img_pil, img_tensor




<p>To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your <code>process_image</code> function works, running the output through this function should return the original image (except for the cropped out portions).</p>


In [ ]:

def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax



In [ ]:

img_path = "flowers/test/17/image_03830.jpg"
img_orig, img_tensor = process_image(img_path)



In [ ]:

img_orig



In [ ]:

imshow(img_tensor)




<h2 id="Class-Prediction">Class Prediction<a class="anchor-link" href="#Class-Prediction">¶</a></h2><p>Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.</p>
<p>To get the top $K$ largest values in a tensor use <a href="http://pytorch.org/docs/master/torch.html#torch.topk"><code>x.topk(k)</code></a>. This method returns both the highest <code>k</code> probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using <code>class_to_idx</code> which hopefully you added to the model or from an <code>ImageFolder</code> you used to load the data (<a href="#Save-the-checkpoint">see here</a>). Make sure to invert the dictionary so you get a mapping from index to class as well.</p>
<p>Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.</p>
<div class="highlight"><pre><span></span><span class="n">probs</span><span class="p">,</span> <span class="n">classes</span> <span class="o">=</span> <span class="n">predict</span><span class="p">(</span><span class="n">image_path</span><span class="p">,</span> <span class="n">model</span><span class="p">)</span>
<span class="k">print</span><span class="p">(</span><span class="n">probs</span><span class="p">)</span>
<span class="k">print</span><span class="p">(</span><span class="n">classes</span><span class="p">)</span>
<span class="o">&gt;</span> <span class="p">[</span> <span class="mf">0.01558163</span>  <span class="mf">0.01541934</span>  <span class="mf">0.01452626</span>  <span class="mf">0.01443549</span>  <span class="mf">0.01407339</span><span class="p">]</span>
<span class="o">&gt;</span> <span class="p">[</span><span class="s1">'70'</span><span class="p">,</span> <span class="s1">'3'</span><span class="p">,</span> <span class="s1">'45'</span><span class="p">,</span> <span class="s1">'62'</span><span class="p">,</span> <span class="s1">'55'</span><span class="p">]</span>
</pre></div>


In [ ]:

def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    model.class_to_idx = train_data.class_to_idx
    img_tensor = process_image(image_path) 
    reshaped = img_tensor[1].unsqueeze(0)

    # handle device
    device = get_device()
    if (str(device) == 'cpu'):
        with torch.no_grad():
            output=model.forward(reshaped)
    else:
        with torch.no_grad():
            output=model.forward(reshaped.cuda())
    
    ps = torch.exp(output)
    probs, classes = ps.topk(topk, dim=1)
    
    # Invert classes
    inverted = {v: k for k, v in model.class_to_idx.items()}
    classes2 = [inverted[c] for c in classes[0].cpu().numpy()]

    return probs, classes2



In [ ]:

probs, classes = predict(img_path, model)
print(probs)
print(classes)




<h2 id="Sanity-Checking">Sanity Checking<a class="anchor-link" href="#Sanity-Checking">¶</a></h2><p>Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use <code>matplotlib</code> to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:</p>
<p><img src="assets/inference_example.png" width="300px"/></p>
<p>You can convert from the class integer encoding to actual flower names with the <code>cat_to_name.json</code> file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the <code>imshow</code> function defined above.</p>


In [ ]:

def check_sanity(image_path, model):
    prob, classes = predict(image_path, model)
    max_index = np.argmax(prob)
    max_probability = prob[max_index]
    label = classes[max_index]

    fig = plt.figure(figsize=(6,6))
    ax1 = plt.subplot2grid((15,9), (0,0), colspan=9, rowspan=9)
    ax2 = plt.subplot2grid((15,9), (9,2), colspan=5, rowspan=10)

    #image = Image.open(img_path)
    img_orig, img_tensor = process_image(image_path)
    names = [cat_to_name[c] for c in classes]
    
    ax1.axis('off')
    ax1.set_title(cat_to_name[label])
    ax1.imshow(img_orig)
    
    y_pos = np.arange(5)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(names)
    ax2.set_xlabel('Probability')
    ax2.invert_yaxis()
    ax2.barh(y_pos, prob[0], xerr=0, align='center', color='green')

    plt.show()



In [ ]:

check_sanity(img_path, model)

